In [1]:
#Lab2
#Marek Fudaliński

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es = Elasticsearch(HOST="http://localhost", PORT=9200)
es = Elasticsearch()
index = "my-index-000001"

In [4]:
es.indices.delete(index=index)

{'acknowledged': True}

In [5]:
#Defin analyzer
es.indices.create(
    index = index,
    body = {
        "settings": {
            "analysis": {
                "analyzer": {
                    "polish": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "synonym",
                            "morfologik_stem"
                        ]
                    }
                },
                "filter": {
                    "synonym": {
                        "type": "synonym",
                        "tokenizer": "standard",
                        "synonyms": [
                            "kpk, kodeks postępowania karnego",
                            "kpc, kodeks postępowania cywilnego",
                            "kk, kodeks karny",
                            "kc, kodeks cywilny"
                        ]
                    }
                }
            }
        },
        "mappings": {
            "dynamic": "strict",
            "properties": {
                "bill": {
                    "type": "text",
                    "analyzer": "polish",
                },
                "file_name": {"type": "text"}
             }
        }
    }
)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'my-index-000001'}

In [6]:
from os import listdir
from os.path import isfile, join
billsPath = './bills/'
onlyBills = [f for f in listdir(billsPath) if isfile(join(billsPath, f))]
print(len(onlyBills))

1180


In [7]:
for i in range(len(onlyBills)):
    f = open(billsPath+onlyBills[i],"r",encoding='utf-8')
    text = f.read()
    body = {"bill":text,"file_name":onlyBills[i]}
    f.close()
    es.index(index=index, id=i, body=body)

In [8]:
# 6. Determine the number of legislative acts containing the word ustawa (in any form)
body = {
    "query": {
        "match": {
            "bill":"ustawa"
        }
    }
}
res = es.count(index=index, body=body)
res

{'count': 1179,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [9]:
# 7 Determine the number of legislative acts containing the words kodeks postępowania cywilnego in the specified order, but in an any inflection form.
body = {
    "query": {
        "match_phrase": {
            "bill": {
                "query": "kodeks postępowania cywilnego",
#                 "slop": 0
            }
        }
    }
}
res = es.count(index=index, body=body)
res


{'count': 100,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [10]:
# 8 Determine the number of legislative acts containing the words wchodzi w życie (in any form) allowing for up to 2 additional words in the searched phrase.
body = {
    "query": {
        "match_phrase": {
            "bill": {
                "query": "wchodzi w życie",
                "slop": 2
            }
        }
    }
}
res = es.count(index=index, body=body)
res


{'count': 1175,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [11]:
#9 Determine the 10 documents that are the most relevant for the phrase konstytucja.
body = {
    "size":10,
    "_source": ['file_name'],
    "query": {
        "match": {
            "bill":"konstytucja"
        },
    }
}

res = es.search(index=index, body=body)
res
for i in range(len(res['hits']['hits'])):
    print(str(i + 1) +". " + res['hits']['hits'][i]['_source']['file_name'] + " : " + str(res['hits']['hits'][i]['_score']))

1. 1997_629.txt : 6.87158
2. 2000_443.txt : 6.667763
3. 1997_604.txt : 6.637804
4. 1996_350.txt : 6.631998
5. 1997_642.txt : 6.2563715
6. 2001_23.txt : 6.0651627
7. 1996_199.txt : 5.935479
8. 1999_688.txt : 5.8563156
9. 1997_681.txt : 5.473405
10. 2001_1082.txt : 5.473405


In [12]:
#10 Print the excerpts containing the word konstytucja (up to three excerpts per document) from the previous task.
body = {
    "query": {
        "match": {
            "bill": {
                "query": "konstytucja"
            }
        }
    },
    "size": 10,
    "_source": [ "file_name" ],
    "highlight": {
        "number_of_fragments": 3,
        "fragment_size": 100,
        "fields": {
            "bill": {}
        }
    }
}
res = es.search(index=index, body=body)
res['hits']['hits']
for i in range(len(res['hits']['hits'])):
    print(str(i + 1) + ') ' + res['hits']['hits'][i]['_source']['file_name'])
    for j in range(len(res['hits']['hits'][i]['highlight']['bill'])):
        print("\t" + str(j+1) + ". " + res['hits']['hits'][i]['highlight']['bill'][j])

1) 1997_629.txt
	1. o zmianie ustawy konstytucyjnej o trybie przygotowania
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej
	2. W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i 
uchwalenia <em>Konstytucji</em>
	3. Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz 
                obywateli popierających zgłoszenie
2) 2000_443.txt
	1. umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89
     ust. 1 lub art. 90 <em>Konstytucji</em>
	2. międzynarodowej lub załącznika nie
     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>
	3. co do zasadności wyboru
  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2
  <em>Konstytucji</em>
3) 1997_604.txt
	1. Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności celów partii 
   politycznej z <em>Konstytucją</em>
	2. Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności z <em>Konstytucją</em>
   celów lub działalno